In [1]:
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install statsmodels
# !{sys.executable} -m pip install tabulate
# !{sys.executable} -m pip install httpimport

import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.iolib.summary2 import summary_col
from tabulate import tabulate

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_panel_diff = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data_election_crime_merged.csv')

In [3]:
data_panel_diff = pd.concat([data_panel_diff, pd.get_dummies(data_panel_diff['party'], prefix='party')],axis=1)
data_panel_diff = pd.concat([data_panel_diff, pd.get_dummies(data_panel_diff['bl_kuerzel'], prefix='bl')],axis=1)

In [4]:
data_panel_diff.columns

Index(['Unnamed: 0', 'crime', 'city', 'law', 'date', 'background', 'suspects',
       'party', 'state', 'plz', 'city_id', 'bl_kuerzel', 'year', 'month',
       'day', 'cycle_1', 'cycle_2', 'cycle_3', 'crime_count',
       'Anzahl Bewerber', 'darunter Frauen', 'Land', 'crime_count_party_lr',
       'crime_count_party', 'crime_count_lr', 'crime_count_city', 'party_lr',
       'Bewerber_city', 'Frauen_city', 'bewerber_hit_by_lr', 'fem_hit_by_lr',
       'bewerber_party_hit_by_lr', 'fem_party_hit_by_lr', 'p_female',
       'p_female_lr', 'p_female_city', 'p_female_party_lr', 'p_female_diff',
       'p_female_city_diff', 'p_female_lr_diff', 'p_female_party_lr_diff',
       'county', 'county.1', 'party_AFD', 'party_AfD', 'party_AfD ',
       'party_B 90/Grüne', 'party_B90/GRÜNE', 'party_B90/Grüne', 'party_CDU',
       'party_CDU ', 'party_DIE LINKE', 'party_DIE LINKE ', 'party_DIE LINKE.',
       'party_DIE PARTEI', 'party_DKP', 'party_DSU', 'party_Die LINKE',
       'party_Die Linke', 'part

### Let's consider the standard measure of crime (against a party in a city)

In [32]:
data_panel_diff = data_panel_diff[data_panel_diff['cycle_1'] == 2014]
data_panel_diff = data_panel_diff.drop_duplicates(['city', 'party', 'plz', 'state', 'p_female_diff', 'crime_count_party'], ignore_index=True)
data_panel_diff

,Unnamed: 0,crime,city,law,date,background,suspects,party,state,plz,...,party_AfD,party_CDU,party_DIE LINKE,party_FDP,party_GRÜNE,party_SPD,bl_BB,bl_BW,bl_SN,bl_TH
0,17,Beleidigung,Stuttgart,185 StGB,2019-01-09,Links,1.0,AfD,BW,70173.0,...,1,0,0,0,0,0,0,1,0,0
1,19,NaN,Stuttgart,NaN,NaN,NaN,NaN,CDU,BW,70173.0,...,0,1,0,0,0,0,0,0,0,0
2,20,NaN,Stuttgart,NaN,NaN,NaN,NaN,DIE LINKE,BW,70173.0,...,0,0,1,0,0,0,0,0,0,0
3,21,NaN,Stuttgart,NaN,NaN,NaN,NaN,FDP,BW,70173.0,...,0,0,0,1,0,0,0,0,0,0
4,22,NaN,Stuttgart,NaN,NaN,NaN,NaN,GRÜNE,BW,70173.0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8967,20227,NaN,Pinnow,NaN,NaN,NaN,NaN,DIE LINKE,BB,16278.0,...,0,0,1,0,0,0,0,0,0,0
8968,20228,NaN,Pinnow,NaN,NaN,NaN,NaN,CDU,BB,16278.0,...,0,1,0,0,0,0,0,0,0,0
8969,20229,NaN,Pinnow,NaN,NaN,NaN,NaN,GRÜNE,BB,16278.0,...,0,0,0,0,1,0,0,0,0,0
8970,20233,NaN,Passow,NaN,NaN,NaN,NaN,CDU,BB,16306.0,...,0,1,0,0,0,0,0,0,0,0


In [33]:
data_panel_diff['p_female_diff'] = data_panel_diff['p_female_diff']*100

In [34]:
data_panel_diff['p_female_diff'].mean()

1.8683891691055903

In [35]:
data_panel_diff[['p_female_diff', 'city', 'crime_count_party']]

,p_female_diff,city,crime_count_party
0,6.666667,Stuttgart,14.0
1,-5.000000,Stuttgart,1.0
2,1.666667,Stuttgart,0.0
3,-1.666667,Stuttgart,4.0
4,0.000000,Stuttgart,1.0
...,...,...,...
8967,20.000000,Pinnow,0.0
8968,5.555556,Pinnow,0.0
8969,0.000000,Pinnow,0.0
8970,0.000000,Passow,0.0


In [36]:
print("Conditional means based on party (no restriction to wether a crime has happened or whether the party stood twice)")
data_panel_diff.groupby(['party'], as_index=False).mean()[['party', 'crime_count_party', 'p_female_diff']], len(data_panel_diff[~data_panel_diff['p_female_diff'].isna()])

Conditional means based on party (no restriction to wether a crime has happened or whether the party stood twice)


(       party  crime_count_party  p_female_diff
 0        AfD           0.348513       8.293668
 1        CDU           0.083969       1.369342
 2  DIE LINKE           0.000000       2.868536
 3        FDP           0.034605       1.608198
 4      GRÜNE           0.075806       3.159392
 5        SPD           0.163842       1.339538, 3843)

In [37]:
# Drop missing values
data_panel_diff.dropna(subset=['p_female_diff', 'crime_count_party'], inplace=True)
print("Conditional means based on party when they stood for office twice")
data_panel_diff.groupby(['party'], as_index=False).mean()[['party', 'crime_count_party', 'p_female_diff']], len(data_panel_diff[~data_panel_diff['p_female_diff'].isna()])

Conditional means based on party when they stood for office twice


(       party  crime_count_party  p_female_diff
 0        AfD           5.583333       8.293668
 1        CDU           0.106242       1.369342
 2  DIE LINKE           0.000000       2.868536
 3        FDP           0.121951       1.608198
 4      GRÜNE           0.217507       3.159392
 5        SPD           0.266667       1.339538, 3843)

In [38]:
X = data_panel_diff['crime_count_party']
y = data_panel_diff['p_female_diff']
X = sm.add_constant(X)
model1 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     5.800
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.251
Time:                        10:45:54   Log-Likelihood:                -16512.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3841   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.8567      0.278      6.687      0.000       1.313       2.401
crime_count_party     0.0553      0.023      2.408      0.016       0.010       0.100
==============================================================================
Omnibus:                      625.666   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5603.544
Skew:                           0.502   Prob(JB):                         0.00
Kurtosis:                       8.830   Cond. No.                         2.09
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [39]:
model2 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party']})
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     3.065
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.140
Time:                        10:45:54   Log-Likelihood:                -16512.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3841   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.8567      0.346      5.364      0.000       1.178       2.535
crime_count_party     0.0553      0.032      1.751      0.080      -0.007       0.117
==============================================================================
Omnibus:                      625.666   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5603.544
Skew:                           0.502   Prob(JB):                         0.00
Kurtosis:                       8.830   Cond. No.                         2.09
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [40]:
model3 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['state']})
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     3.345
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.165
Time:                        10:45:55   Log-Likelihood:                -16512.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3841   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.8567      0.720      2.578      0.010       0.445       3.268
crime_count_party     0.0553      0.030      1.829      0.067      -0.004       0.114
==============================================================================
Omnibus:                      625.666   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5603.544
Skew:                           0.502   Prob(JB):                         0.00
Kurtosis:                       8.830   Cond. No.                         2.09
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [56]:
print(summary_col([model1,model2, model3]))
# A increase by one crime against a party is associated with an 0.0005% higher share of females in the following period. 


                  p_female_diff I p_female_diff II p_female_diff III
--------------------------------------------------------------------
const             1.8567          1.8567           1.8567           
                  (0.2776)        (0.3461)         (0.7202)         
crime_count_party 0.0553          0.0553           0.0553           
                  (0.0229)        (0.0316)         (0.0302)         
Standard errors in parentheses.


In [42]:
data_panel_diff2 = data_panel_diff.drop_duplicates(['city', 'plz', 'state', 'p_female_city_diff', 'crime_count_city'], ignore_index=True)
data_panel_diff2.dropna(subset=['p_female_city_diff', 'crime_count_city'], inplace=True)

X2 = data_panel_diff2["crime_count_city"]
y = data_panel_diff2["p_female_city_diff"]
X2 = sm.add_constant(X2)
model4 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff2['party_lr']})
model4.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     p_female_city_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2234
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.719
Time:                        10:45:55   Log-Likelihood:                 1144.8
No. Observations:                1666   AIC:                            -2286.
Df Residuals:                    1664   BIC:                            -2275.
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0037      0.006      0.649      0.516      -0.007       0.015
crime_count_city  2.748e-05   5.81e-05      0.473      0.636   -8.65e-05       0.000
==============================================================================
Omnibus:                      289.746   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2742.155
Skew:                          -0.517   Prob(JB):                         0.00
Kurtosis:                       9.199   Cond. No.                         5.69
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [43]:
X2 = data_panel_diff2["crime_count_city"]
y = data_panel_diff2["p_female_city_diff"]
X2 = sm.add_constant(X2)
model5 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff2['party']})
model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     p_female_city_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.02033
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.892
Time:                        10:45:55   Log-Likelihood:                 1144.8
No. Observations:                1666   AIC:                            -2286.
Df Residuals:                    1664   BIC:                            -2275.
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0037      0.004      0.889      0.374      -0.004       0.012
crime_count_city  2.748e-05      0.000      0.143      0.887      -0.000       0.000
==============================================================================
Omnibus:                      289.746   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2742.155
Skew:                          -0.517   Prob(JB):                         0.00
Kurtosis:                       9.199   Cond. No.                         5.69
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [44]:
data_panel_lr = data_panel_diff.drop_duplicates(['city', 'party', 'plz', 'state', 'p_female_party_lr', 'crime_count_party_lr'], ignore_index=True)
data_panel_lr.dropna(subset=['p_female_party_lr', 'crime_count_party_lr'], inplace=True)

X2 = data_panel_lr["crime_count_party_lr"]
y = data_panel_lr["p_female_party_lr"]
X2 = sm.add_constant(X2)
model6 = sm.OLS(y, X2).fit(cov_type='HC3')
model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      p_female_party_lr   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.3475
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.562
Time:                        10:45:55   Log-Likelihood:                 10.978
No. Observations:                  24   AIC:                            -17.96
Df Residuals:                      22   BIC:                            -15.60
Df Model:                           1                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.2614      0.040      6.609      0.000       0.184       0.339
crime_count_party_lr    -0.0013      0.002     -0.589      0.556      -0.006       0.003
==============================================================================
Omnibus:                        1.365   Durbin-Watson:                   1.578
Prob(Omnibus):                  0.505   Jarque-Bera (JB):                0.938
Skew:                          -0.127   Prob(JB):                        0.626
Kurtosis:                       2.066   Cond. No.                         23.5
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [47]:
%run estout_func.py

In [49]:
X3 = data_panel_diff[["crime_count_party", "party_GRÜNE", "party_SPD", "party_CDU", "party_FDP", "party_DIE LINKE", 'bl_BB', 'bl_SN', 'bl_TH']]
y2 = data_panel_diff["p_female_diff"]
X3 = sm.add_constant(X3)
model7 = sm.OLS(y2, X3).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model7.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                    0.3085
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.677
Time:                        10:46:27   Log-Likelihood:                -16506.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3833   BIC:                         3.309e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 8.6683      0.817     10.611      0.000       7.067      10.270
crime_count_party    -0.0885      0.055     -1.615      0.106      -0.196       0.019
party_GRÜNE          -5.4838      0.791     -6.929      0.000      -7.035      -3.933
party_SPD            -7.3126      0.801     -9.124      0.000      -8.883      -5.742
party_CDU            -7.2926      0.812     -8.984      0.000      -8.884      -5.702
party_FDP            -7.0493      0.810     -8.700      0.000      -8.637      -5.461
party_DIE LINKE      -5.7998      0.817     -7.099      0.000      -7.401      -4.199
bl_BB                -3.9952      7.814     -0.511      0.609     -19.310      11.319
bl_SN                 2.7423      1.781      1.539      0.124      -0.749       6.234
bl_TH                -0.9829      4.459     -0.220      0.826      -9.723       7.757
==============================================================================
Omnibus:                      589.236   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5420.436
Skew:                           0.438   Prob(JB):                         0.00
Kurtosis:                       8.752   Cond. No.                         76.1
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [58]:
model8 = sm.OLS(y2, X3).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['state']})
model8.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 4
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                -5.411e+13
Date:                Thu, 23 Sep 2021   Prob (F-statistic):               1.00
Time:                        16:07:41   Log-Likelihood:                -16506.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3833   BIC:                         3.309e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 8.6683      1.921      4.512      0.000       4.903      12.433
crime_count_party    -0.0885      0.051     -1.741      0.082      -0.188       0.011
party_GRÜNE          -5.4838      1.985     -2.763      0.006      -9.374      -1.594
party_SPD            -7.3126      1.706     -4.286      0.000     -10.656      -3.969
party_CDU            -7.2926      1.446     -5.044      0.000     -10.126      -4.459
party_FDP            -7.0493      1.241     -5.678      0.000      -9.483      -4.616
party_DIE LINKE      -5.7998      1.635     -3.547      0.000      -9.005      -2.595
bl_BB                -3.9952      1.570     -2.545      0.011      -7.072      -0.918
bl_SN                 2.7423      1.548      1.772      0.076      -0.292       5.776
bl_TH                -0.9829      0.938     -1.048      0.295      -2.821       0.856
==============================================================================
Omnibus:                      589.236   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5420.436
Skew:                           0.438   Prob(JB):                         0.00
Kurtosis:                       8.752   Cond. No.                         76.1
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [59]:
model9 = sm.OLS(y2, X3).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party']})
model9.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 4
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2851.
Date:                Thu, 23 Sep 2021   Prob (F-statistic):           1.41e-08
Time:                        16:07:46   Log-Likelihood:                -16506.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3833   BIC:                         3.309e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 8.6683      0.579     14.976      0.000       7.534       9.803
crime_count_party    -0.0885      0.064     -1.392      0.164      -0.213       0.036
party_GRÜNE          -5.4838      0.554     -9.905      0.000      -6.569      -4.399
party_SPD            -7.3126      0.565    -12.953      0.000      -8.419      -6.206
party_CDU            -7.2926      0.573    -12.719      0.000      -8.416      -6.169
party_FDP            -7.0493      0.572    -12.318      0.000      -8.171      -5.928
party_DIE LINKE      -5.7998      0.579    -10.020      0.000      -6.934      -4.665
bl_BB                -3.9952      5.988     -0.667      0.505     -15.731       7.740
bl_SN                 2.7423      2.264      1.211      0.226      -1.696       7.181
bl_TH                -0.9829      2.988     -0.329      0.742      -6.840       4.874
==============================================================================
Omnibus:                      589.236   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5420.436
Skew:                           0.438   Prob(JB):                         0.00
Kurtosis:                       8.752   Cond. No.                         76.1
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [60]:
estout_ols(modellist=[model1, model2, model3, model7, model8, model9], modellist_str = ["model1","model2","model3","model4","model5","model6"], y='Share Female Candidates', caption="Basic Regression", label="Basic Regression", list_regression_sets=[X,X,X,X3,X3,X3], p_values=True)

\begin{table}[htbp] \caption{Basic Regression \label{Basic Regression}}
\resizebox{0.9\textwidth}{!}{ \centering
\begin{tabular}{lcccccc} \hline
  & (I) & (II) & (III) & (IV) & (V) & (VI) \\ 
Dependent Variable & \multicolumn{6}{c}{Share Female Candidates} \\ \hline \vspace{4pt} 
& \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize}  \\ 
bl_BB &  &  &  & -3.9952** & -3.9952** & -3.9952 \\ 
 \vspace{4pt} & \begin{footnotesize} \end{footnotesize} & \begin{footnotesize} \end{footnotesize} & \begin{footnotesize} \end{footnotesize} & \begin{footnotesize}(1.5699) \end{footnotesize} & \begin{footnotesize}(1.5699) \end{footnotesize} & \begin{footnotesize}(5.9877) \end{footnotesize} \\ 
bl_SN &  &  &  & 2.7423* & 2.7423* & 2.7423 \\ 
 \vspace{4pt} & \begin{footnotesize} \end{footnotesize} & \begin{footnote

In [52]:
data_panel_diff_balanced_noafd = data_panel_diff[data_panel_diff['party_AfD'] != 1]
X4 = data_panel_diff_balanced_noafd[["crime_count_party", "party_SPD", "party_FDP", "party_CDU", "party_GRÜNE", 'bl_BB', 'bl_SN', 'bl_TH']]
y3 = data_panel_diff_balanced_noafd["p_female_diff"]
X4 = sm.add_constant(X4)
model9 = sm.OLS(y3, X4).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff_balanced_noafd['party_lr']})
model9.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1364
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.775
Time:                        10:46:27   Log-Likelihood:                -16261.
No. Observations:                3795   AIC:                         3.254e+04
Df Residuals:                    3786   BIC:                         3.260e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.8685   7.59e-15   3.78e+14      0.000       2.869       2.869
crime_count_party     0.0791      0.060      1.321      0.187      -0.038       0.197
party_SPD            -1.5564      0.013   -119.600      0.000      -1.582      -1.531
party_FDP            -1.2700      0.007   -173.845      0.000      -1.284      -1.256
party_CDU            -1.5100      0.004   -370.052      0.000      -1.518      -1.502
party_GRÜNE           0.2531      0.002    118.937      0.000       0.249       0.257
bl_BB                 6.7201      0.118     57.106      0.000       6.489       6.951
bl_SN                 2.5831      0.553      4.672      0.000       1.500       3.667
bl_TH                -1.5560      4.545     -0.342      0.732     -10.464       7.352
==============================================================================
Omnibus:                      562.218   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5273.699
Skew:                           0.399   Prob(JB):                         0.00
Kurtosis:                       8.720   Cond. No.                         71.3
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [53]:
data_panel_diff_balanced_noafd = data_panel_diff[data_panel_diff['party_GRÜNE'] != 1]
X5 = data_panel_diff_balanced_noafd[["crime_count_party", "party_SPD", "party_FDP", "party_CDU", "party_AfD", 'bl_BB', 'bl_SN', 'bl_TH']]
y3 = data_panel_diff_balanced_noafd["p_female_diff"]
X5 = sm.add_constant(X5)
model10 = sm.OLS(y3, X5).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff_balanced_noafd['party_lr']})
model10.summary()

C:\Users\mariu\Anaconda3\lib\site-packages\statsmodels\base\model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                   0.05010
Date:                Thu, 23 Sep 2021   Prob (F-statistic):              0.860
Time:                        10:46:27   Log-Likelihood:                -14724.
No. Observations:                3466   AIC:                         2.947e+04
Df Residuals:                    3457   BIC:                         2.952e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 2.8685   1.59e-14    1.8e+14      0.000       2.869       2.869
crime_count_party    -0.1370      0.032     -4.227      0.000      -0.201      -0.073
party_SPD            -1.5039      0.002   -603.238      0.000      -1.509      -1.499
party_FDP            -1.2436      0.004   -314.684      0.000      -1.251      -1.236
party_CDU            -1.4896      0.002   -747.494      0.000      -1.494      -1.486
party_AfD             6.2138      0.227     27.392      0.000       5.769       6.658
bl_BB                -9.8544      0.259    -38.011      0.000     -10.363      -9.346
bl_SN                 3.7134      3.811      0.974      0.330      -3.756      11.183
bl_TH                 0.0521      5.438      0.010      0.992     -10.606      10.711
==============================================================================
Omnibus:                      521.320   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4476.586
Skew:                           0.445   Prob(JB):                         0.00
Kurtosis:                       8.496   Cond. No.                         91.6
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

---